# Mott transition on the Bethe lattice Hubbard model

In this notebook you will reproduce the Bethe lattice DMFT that we did earlier with IPT,
but you will use the CT-INT solver to find the solution of the impurity problem. We will
consider the problem at half-filling again. You will see that the numerically
exact result is a bit different from IPT. This can be seen in the 
plot below that shows the Mott transition obtained by the iterated perturbation
theory and by the numerical renormalzation group. It is taken from
the article
_Finite-temperature numerical renormalization group study of the Mott transition_,
R. Bulla, T. A. Costi, and D. Vollhardt
Phys. Rev. B 64, 045103 (2001). Note that here the unit are $W = 4t$, where $W$ is the full bandwith of the
semicircular.

<img src="figures/dmft_bethe.png" alt="2d_hubbard" style="width: 400px;"/>


## Exercise 1

Write a DMFT loop, like you did earlier but this time using the CT-INT solver. Solve the Bethe lattice
model at $\beta=10$ for $U=4.0$ and $U=5.0$. Try to save your results to an archive in order to
postprocess them later.

*Hint*: It is useful to symmetrize the Green's function (make the `up` and `down` components the same) to avoid some artificial polarization of the system close to the Mott transition. You might want to enforce the `up`-`down` symmetry on `S.G` just before the self-consistency condition. In order
to have reasonable data you should have at least 10000 cycles. You will see that simulations take some time.

Exercise 2
----------

Here, you will learn to analyze the output of the solver. As discussed, the Monte Carlo algorithm provide results on the
Matsubara axis. This makes the analysis of the results slightly more delicate than if we had them directly on the real
axis. When we used the IPT solver, we could see the Mott transition as the appearance of a gap in the spectral function.
After the Monte Carlo run, we do not have the spectral function, so we will have to use some other criteria to decide, e.g.,
if the system is metallic or insulating.

Plot the Green's function at the end of the DMFT loops for different values of $U$ (say 4 and 5). Focus on the extrapolation of the imaginary part of the Green's function to zero frequency. How does it change with $U$? Is there
a way to see the Mott transition just by inspecting the imaginary part of the Green's function?

## Exercise 3

Do the same exercise as above, but analyze the self-energy. The noise usually gets bigger for larger frequencies, so just focus on the first few Matsubara frequencies. There the noise should not be too important. Again, by looking at the extrapolation to zero frequency of the imaginary part of the self-energy, can you tell where the Mott transition happens?

## Exercise 4

A very useful quantity to measure the degree of correlation of a metal is the *quasiparticle weight* $Z$. It is defined as

$$
Z = \lim_{\omega_n \rightarrow 0} \Big( 1 - \frac{d\mathrm{Im} \Sigma(i\omega_n)}{di\omega_n} \Big)^{-1}
$$

For a non-interacting metal $Z=1$. As correlations appear, $Z$ gradually gets smaller. It reaches 0 at the Mott transition. Compare $Z$ for the two values of $U$ that you computed above. If you have more time,
make a plot of $Z$ versus $U$ for the Bethe lattice Hubbard model.

*Hint*: In order to have access to the values of $\Sigma_\uparrow(i\omega_n)$, you can use `Sigma['up'](n)`. This will be useful to numerically compute the derivative required to compute $Z$.

Exercise 5
----------

Go back to your IPT code and try to modify it to extract the $Z$ versus $U$ curve. Compare this to the result you found in Exercise 4. Is the critical $U$ for the Mott transition similar to the one you found using CT-INT? Is
it compatible with the plot above?

Exercise 6
----------

Try to analytically continue the Green's function on the real axis using the Pade approximation. What can
you say about the result?